In [1]:
import os, sys, joblib, glob, shutil, json, time, datetime
import cv2
#os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import darknet
import requests
from collections import Counter
import threading, multiprocessing
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn.model import log
import imageio, gc

gpus = tf.config.experimental.list_physical_devices('GPU')
if len(gpus)>0:
    try:
        tf.config.experimental.set_virtual_device_configuration(gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1000)])
#         tf.config.experimental.set_memory_growth(gpus[0], True) 
#         tf.config.experimental.set_memory_growth(gpus[1], True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
else:
    print('No GPU')


1 Physical GPUs, 1 Logical GPUs


In [2]:
def checkFolderExist(path):
    if not os.path.exists(path):
        os.makedirs(path)
        
def getPreviewUrl(cameraName):
    url = 'http://localhost:8081/v1/api/ipcamera/previewurl/name'
#     url = 'http://10.142.3.61:8081/v1/api/ipcamera/previewurl/name'
#    url = 'http://10.109.6.148:8081/v1/api/ipcamera/replayurl/name'     
    data = {
        "cameraName":cameraName,
        "expand":"streamform=rtp" ,
        #"transcode=1&resolution=D1&bitrate=512&framerate=15&streamform=rtp&snapshot=1"
    }
    data_json = json.dumps(data)
    headers = {'Content-type': 'application/json'}

    response = requests.post(url, data=data_json, headers=headers)
    jsonObject = response.json()
    replayUrl = ""
    if jsonObject['code'] == '200':
        replayUrl = jsonObject['result']['replayUrl']
    #print(replayUrl)
    print(jsonObject)
    return replayUrl


def getReplayUrl(cameraName, t1_str, t2_str):
    url = 'http://localhost:8081/v1/api/ipcamera/replayurl/name'
    data = {
        "cameraName":cameraName, 
        "beginTime":t1_str, 
        "endTime":t2_str,
        "expand":"streamform=rtp",        
#        "expand":"transcode=1&resolution=D1&bitrate=1024&framerate=15&streamform=rtp"
    }
    #print(f'{cameraName}: {t1_str} ~ {t2_str}')
    data_json = json.dumps(data)
    headers = {'Content-type': 'application/json'}

    response = requests.post(url, data=data_json, headers=headers)
    jsonObject = response.json()
    print(jsonObject)
    replayUrl = ""
    if jsonObject['code'] == '200':
        replayUrl = jsonObject['result']['replayUrl']
    print(replayUrl)
    return replayUrl

def checkImageNone(fid1, cameraName, cap):
    print('last none fid:', fid1, datetime.datetime.today())
    time.sleep(60)
    replayUrl = getPreviewUrl(cameraName)
    cap = cv2.VideoCapture(replayUrl)
    ret, image = cap.read()
    return ret, image


## YOLO

In [3]:
weights_path = 'model/yolov4-tiny-f45-0507_best.weights'#模型权重文件
cfg_path = 'cfg/yolov4-tiny-f45-0331.cfg'#模型配置文件
labels_path = 'cfg/f45_obj.names'#模型类别标签文件

# class: color
classid_dict = {0: ['battery_p',(127,255,50)],
                1: ['battery_f',(147,20,255)],
                2: ['battery',(135,138,128)],
                3: ['vpen',(153, 255,255)],
                4: ['wz',(221,160,221)],
                5: ['ppen',(255,255,86)]}

def yolo_dnn(image):
    (H,W) = image.shape[: 2]    
    # construct a blob from the input frame and then perform a forward
    # pass of the YOLO object detector, giving us our bounding boxes
    # and associated probabilities
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    layerOutputs = net.forward(ln)  
    boxes = []
    confidences = []
    classIDs = []
    for output in layerOutputs:  # loop over each of the layer outputs
        for detection in output:  # loop over each of the detections
            # extract the class ID and confidence (i.e., probability) of the current object detection
            scores=detection[5:]  #detection=[x,y,h,w,c,class1,class2] scores取第6位至最后
            classID = np.argmax(scores)#np.argmax反馈最大值的索引
            confidence = scores[classID]
            if confidence >0.2: # filter out weak predictions by ensuring the detected probability is greater than the minimum probability
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height)= box.astype("int")
                # left-up corner
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                # update
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)
    idxs=cv2.dnn.NMSBoxes(boxes, confidences, 0.2, 0.3)
    detections = []
    if len(idxs)>0:
        box_seq = idxs.flatten()    
        for seq in box_seq:
            detections.append([classIDs[seq], round(confidences[seq],2), boxes[seq]])
    return detections

def yolo_bat(image, detections, bat_score):
    bat_img = None
    f_score = 0
    battery_p = list(filter(lambda x: x[0]==0, detections))  #battery_p
    if len(battery_p) > 0:
        return bat_img, f_score
    battery_f = list(filter(lambda x: x[0]==1, detections))  #battery_f
    battery_f = sorted(battery_f, key = lambda x: float(x[1]), reverse=True)
    battery_f = list(filter(lambda x: float(x[1])>=bat_score, battery_f))
    if len(battery_f) > 0:
        f_score = float(battery_f[0][1])
        x, y, w, h = int(battery_f[0][2][0]), int(battery_f[0][2][1]), int(battery_f[0][2][2]), int(battery_f[0][2][3])
        bat_img = cv2.resize(image[y:y+h, x:x+w], (64,64))
    return bat_img, f_score

def draw_bbox(image, detections):
    for detection in detections:
        #r_w, r_h = (image.shape[1]/net_wh[0], image.shape[0]/net_wh[1])
        x, y, w, h = int(detection[2][0]), int(detection[2][1]), int(detection[2][2]), int(detection[2][3])
#         x=x*r_w
#         w=w*r_w
#         y=y*r_h
#         h=h*r_h
        x1 = int(round(x - (w / 2)))
        y1 = int(round(y - (h / 2)))
        x2 = int(round(x + (w / 2)))
        y2 = int(round(y + (h / 2))) 
        #原始的中心點
        cx, cy = (x1+x2)//2, (y1+y2)//2
        x1 = np.clip(x1, 0, image.shape[1])
        y1 = np.clip(y1, 0, image.shape[0])
        x2 = np.clip(x2, 0, image.shape[1])
        y2 = np.clip(y2, 0, image.shape[0])
        cv2.rectangle(image, (x, y), (x + w, y + h), classid_dict[detection[0]][1], 1)
        idx = detections.index(detection)
        cv2.putText(image, classid_dict[detection[0]][0]+','+str(detection[1]), (10,130+30*idx) , cv2.FONT_HERSHEY_DUPLEX, 0.6, (255,255,128),1)
    cv2.putText(image, f'FID:{fid}', (10,550), cv2.FONT_HERSHEY_DUPLEX,0.7, (255,255,128), 1)
    return image


#初始化一些参数
LABELS = open(labels_path).read().strip().split("\n")

#load YOLO object detector trained
net = cv2.dnn.readNetFromDarknet(cfg_path, weights_path)

#set CUDA as the preferable backend and target
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

#determine only the *output* layer names that we need from YOLO
ln = net.getLayerNames()
out = net.getUnconnectedOutLayers() 
x = []
for i in out:   
    x.append(ln[i[0]-1])    # i[0]-1    取out中的数字  [200][0]=200  ln(199)= 'yolo_82'
ln=x
# ln  =  ['yolo_82', 'yolo_94', 'yolo_106']  得到 YOLO需要的输出层


## MAskRCNN

In [4]:
# Load Model
class BatteryConfig(Config):
    # Give the configuration a recognizable name
    NAME = "battery"
    
    NUM_CLASSES = 1 + 3

    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    IMAGE_MIN_DIM = 64
    IMAGE_MAX_DIM = 64
    USE_MINI_MASK = False
    RPN_ANCHOR_SCALES = (2, 4, 8, 16, 32)

#config = BatteryConfig()
#config.display()

class InferenceConfig(BatteryConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    USE_MINI_MASK = False

inference_config = InferenceConfig()
inference_config.display()

ROOT_DIR = os.path.abspath("")
MODEL_DIR = os.path.join(ROOT_DIR, "model")

# Recreate the model in inference mode
mask_model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                           model_dir='model')

# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model_path = 'model/maskrcnn_0505v2_best.h5'
#model_path = 'model/maskrcnn_best.h5'

# Load trained weights
print("Loading weights from ", model_path)
mask_model.load_weights(model_path, by_name=True)



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  64
IMAGE_META_SIZE                16
IMAGE_MIN_DIM                  64
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [64 64  3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                  

In [5]:
# classID: [class name, color]
classid_mask = {0: ['BG',(255,255,255)],
                1: ['battery',(135,138,128)],
                2: ['vpen',(153, 255,255)],
                3: ['black',(0,0,0)]}

# functions
def draw_mask(image, boxes, masks, class_ids, score):

    N = boxes.shape[0]
    if not N:
        print("\n*** No instances to display *** \n")
    else:
        assert boxes.shape[0] == masks.shape[-1] == class_ids.shape[0]

    masked_image = image.astype(np.uint32).copy()
    for i in range(N):
        class_id = class_ids[i]
        color = classid_mask[class_id][1]
        # Bounding box
        if not np.any(boxes[i]):
            # Skip this instance. Has no bbox. Likely lost in image cropping.
            continue
        y1, x1, y2, x2 = boxes[i]
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        # Mask
        mask = masks[:, :, i]
        masked_image = apply_mask(masked_image, mask, color)
    mergeimg = np.hstack((image, masked_image))
    return mergeimg

def apply_mask(image, mask, color, alpha=0.5):
    for c in range(3):
        image[:, :, c] = np.where(mask == 1,
                                  image[:, :, c] * (1 - alpha) + alpha * color[c] * 1,
                                  image[:, :, c])
    return image

def draw_maskscore(image, r, d):
    a = dict(zip(r['class_ids'], r['scores']))
    alist = [(classid_mask[key][0], int(a[key]*100)) for key in a.keys()] 
    for value in alist:
        idx = alist.index(value)
        cv2.putText(image, str(value), (10,300+30*idx) , cv2.FONT_HERSHEY_DUPLEX, 0.6, (221,160,221),1)
    dlist = [(key, d[key]) for key in d.keys()] 
    for value in dlist:
        idx = dlist.index(value)
        cv2.putText(image, str(str(value).split(',')[:-1]), (10,400+30*idx) , cv2.FONT_HERSHEY_DUPLEX, 0.6, (221,160,221),1)
    return image

def mask_feature(mask):
    contours, _ = cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key = cv2.contourArea, reverse=True)
    if len(contours) == 0:
        return 0,0,0
    area = cv2.contourArea(contours[0])
    if area == 0:
        return 0,0,0
    M = cv2.moments(contours[0])
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    return cX, cY, area

def get_dist(loc1, loc2):
    dist = np.sqrt( (loc1[0] - loc2[0])**2 + (loc1[1] - loc2[1])**2 )
    return np.round(dist,2)


In [6]:
### API
mid = 'F45_5L2'
fps = 15
camera_df = pd.read_excel(f'doc/camera_list_map.xlsx')
camera_name = camera_df[camera_df['mid']==mid]['camera_name'].values[0]
ip = camera_df[camera_df['mid']==mid]['ip'].values[0]
line = camera_df[camera_df['mid']==mid]['mes_line'].values[0]
station = camera_df[camera_df['mid']==mid]['mes_station'].values[0]
print(mid) 

### Local files
# vpath = '/mnt/hdd1/f45_dev/FA_5L2_video/F45_5L2_2021-05-17T17:51:46.000+08:00_G99FPQHKQ123.mp4'
# vname = vpath.split('/')[-1].split('_')[-2]
# #vpath = 'output/1216/F68_5L2_replay_1608101363.1398787.mp4'
# vname = 'test'

# Preview
vpath = getPreviewUrl(camera_name)
vname = 'preview'

## Replay
# t1_str = '2021-3-25T08:00:00.000+08:00'
# t2_str = '2021-3-25T09:00:00.000+08:00'
# # ret = get_time_diff(t1_str, t2_str,camera_name)
# # t1_str = ret['t1_str_new']
# # t2_str = ret['t2_str_new']
# vpath = getReplayUrl(camera_name, t1_str, t2_str)
# vname = 'replay'

dt= datetime.datetime.today()   
dt_str = f'{dt:%m%d}'
#folder = f'/mnt/hdd1/f45_output/{dt_str}'
folder = f'/mnt/hdd1/f45_dev/FA_5L2_video/fail_image/{dt_str}'
if not os.path.exists(folder):
    os.makedirs(folder)
assert os.path.exists(folder)

# vpath = '/mnt/hdd1/f45_ipqc_issuelist/F45_5L2_2021-01-26T14:08:38.000+08:00_14.mp4'
# idx = vpath.split('_')[-1]

vidcap = cv2.VideoCapture(vpath)
imagelist=[]
success = True
fid = 0
bf_record = 0
bk_record = 0
t_record = datetime.datetime.now()
save_image = None
while success:
    success, image = vidcap.read()
    if image is None:
        print('none image')
        break
    if fid > 108000:
        break
    image = image[:608,:720]
    fid = fid + 1
    fstart = time.time()
    detections = yolo_dnn(image)
    #print(fid, detections)
    bat_img, f_score = yolo_bat(image, detections, 0.3)
    if bat_img is not None:
        results = mask_model.detect([bat_img], verbose=0)
        r = results[0]
        a = sorted(r['class_ids'].tolist())
        if all(x in a for x in [1,2,3]) is False: #確認battery, black, vpen都在
            continue
        d= {}
        for i in range(r['masks'].shape[2]):
            cls_name = classid_mask[r['class_ids'][i]][0]
            if cls_name in d.keys():
                if d[cls_name][2] > r['scores'][i]:
                    continue            
            mask = r['masks'][:,:,i].astype(np.uint8)
            mask*=255
            cX, cY, area = mask_feature(mask)
            if area == 0:
                continue
            d[classid_mask[r['class_ids'][i]][0]]=(cX, cY), area, r['scores'][i]
        if ('vpen' and 'battery' and 'black' in d.keys()) is False:
            continue
        k_score = d['black'][1]
        if k_score < 200: #black的限制
            continue
        if (datetime.datetime.now() - t_record).total_seconds() > 3.5: #是否同一"次"
            now = datetime.datetime.now()
            now_str = now.strftime('%Y-%m-%d-%H-%M-%S.%f')[:-5]        
            jpg_path = os.path.join(folder, f'{now_str}.jpg')
            if save_image is not None:
                multiprocessing.Process(target = cv2.imwrite, args = (jpg_path, save_image)).start()
            bf_record = 0
            bk_record = 0
        t_record = datetime.datetime.now()              
        if (f_score - bf_record)*10 + (k_score - bk_record) <= 0: #是否要存該張frame
            continue
        bf_record = f_score
        bk_record = k_score
        end = time.time()
        seconds = end - fstart   
        fps = np.round(1 / seconds,2)
        # draw image and output
        mergeimg = draw_mask(bat_img, r['rois'], r['masks'], r['class_ids'], r['scores'])
        image[:64,:128] = mergeimg
        image = draw_bbox(image, detections)
        image = draw_maskscore(image, r, d)
        cv2.putText(image, f'FPS:{fps}', (10,530), cv2.FONT_HERSHEY_DUPLEX,0.7, (255,255,128), 1)
        save_image = image
#     end = time.time()
#     seconds = end - fstart           
#     fps = np.round(1 / seconds,2)
#     if len(detections) >0:
#         image = draw_bbox(image, detections)
#     cv2.putText(image, f'FPS:{fps}', (10,530), cv2.FONT_HERSHEY_DUPLEX,0.7, (255,255,128), 1)
    #imagelist.append(image)

# folder = 'output'
# writer = imageio.get_writer(str(idx), format='mp4', mode='I', fps=15)    
# for img in imagelist:
#     writer.append_data(img[:,:,::-1])
# writer.close()
# del imagelist
# gc.collect()                

F45_5L2
{'message': None, 'code': '200', 'result': {'replayUrl': 'rtsp://10.142.81.21:554/openUrl/2fCvCyA', 'beginTime': None, 'endTime': None, 'size': None, 'message': 'success'}}


In [8]:
# print(cv2.getBuildInformation())